## Install sqlite
download and install the sqlite from https://www.sqlite.org/download.html

or if you are using conda env run the command: conda install -c anaconda sqlite

## <font color='blue'> Note </fong>:
#### create new db naming convention: < project_s +  summary + v_version + timestamp >
    

e.g.
'project_s_sample_v0.0_20210221'

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
from sqlalchemy import create_engine

import sys
sys.path.insert(1,'../utils/')
from dtypes_utils import *

In [2]:
import sys
sys.path.insert(1,'../utils/')
db_name = 'project_s_instagram_v1_20210314.db'
table_name_tweet = 'twitter_netflix'
table_name_km = 'kaggle_movies'
table_name_ktv = 'kaggle_tvshows'

conn = sqlite3.connect(db_name) #, detect_types=sqlite3.PARSE_DECLTYPES)
c = conn.cursor()

In [3]:
## check tables in db
check_table_q = """
   SELECT name FROM sqlite_master WHERE type='table'
    """
check_table_df = pd.read_sql(check_table_q.format(table1 = table_name_tweet), conn)
check_table_df

,name
0,twitter_netflix
1,kaggle_movies
2,kaggle_tvshows


### wrapper function
wrapper function to standardize the dtypes

In [4]:
def dtypes_wrapper(df, dtype_df):
    
    for col in df.columns:
        col_type = dtype_df.loc[dtype_df['name']==col, 'type'].values[0]
        
        if col_type == 'int':
            df[col] = df[col].apply(lambda x: convert_to_int(x))
            
        elif col_type == 'float':
            df[col] = df[col].apply(lambda x: convert_to_float(x))
            
        elif col_type == 'varchar' or col_type == 'varchar(32)' or col_type == 'text':
            df[col] = df[col].apply(lambda x: convert_to_string(x))
            
        elif col_type == 'BOOLEAN':
            df[col] = df[col].apply(lambda x: convert_to_boolean(x))

    return df

## <font color='blue'>Query

### <font color='green'> Tweet scripted data

In [5]:
engine = create_engine('sqlite:////home/sushi/Documents/project-s-internal/demo/project_s_prototype_v0.1_20210228.db')

tweet_q = '''
select * from {table}
'''

## get dtypes               
tweet_type_q = """
   PRAGMA table_info({table})
    """

tweet_df= pd.read_sql(tweet_q.format(table = table_name_tweet), con=engine, parse_dates=['time_created'])
tweet_type_df = pd.read_sql(tweet_type_q.format(table = table_name_tweet), con=engine)


## apply wrapper function
tweet_df = dtypes_wrapper(tweet_df, tweet_type_df)

In [6]:
tweet_df.describe()

,user_id,tweet_id,followers_count,statuses_count,reply_count,retweet_count,favorites_count
count,7.124200e+04,7.124200e+04,7.124200e+04,7.124200e+04,71242.000000,0.0,5714.0
mean,5.966634e+17,1.365787e+18,6.082730e+03,3.485574e+04,0.248772,NaN,0.0
std,5.846309e+17,2.011390e+13,1.351814e+05,8.169296e+04,5.217174,NaN,0.0
min,6.825830e+05,1.365736e+18,0.000000e+00,1.000000e+00,0.000000,NaN,0.0
25%,5.247963e+08,1.365773e+18,8.900000e+01,2.283000e+03,0.000000,NaN,0.0
50%,7.605843e+17,1.365791e+18,3.210000e+02,9.766000e+03,0.000000,NaN,0.0
75%,1.209161e+18,1.365803e+18,1.033000e+03,3.234100e+04,0.000000,NaN,0.0
max,1.365814e+18,1.365814e+18,1.096955e+07,1.846996e+06,332.000000,NaN,0.0


In [7]:
tweet_df.dtypes

user_id                        float64
tweet_id                       float64
time_created       datetime64[ns, UTC]
reduced_text                    object
full_text                       object
location                        object
followers_count                float64
statuses_count                 float64
verified                          bool
description                     object
reply_count                    float64
retweet_count                  float64
favorites_count                float64
retweeted                       object
hashtags                        object
dtype: object

In [8]:
tweet_df.time_created.min(), tweet_df.time_created.max()

(Timestamp('2021-02-27 18:49:11+0000', tz='UTC'),
 Timestamp('2021-02-27 23:59:59+0000', tz='UTC'))